# Installation


In [1]:
# Clone github files from officials
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git
# Progress bar
!pip install tqdm

  Cloning https://github.com/sberbank-ai/Real-ESRGAN.git to /tmp/pip-req-build-9odg8imd
  Running command git clone --filter=blob:none --quiet https://github.com/sberbank-ai/Real-ESRGAN.git /tmp/pip-req-build-9odg8imd
  Resolved https://github.com/sberbank-ai/Real-ESRGAN.git to commit 362a0316878f41dbdfbb23657b450c3353de5acf
  Preparing metadata (setup.py) ... done
  Created wheel for RealESRGAN: filename=RealESRGAN-1.0-py3-none-any.whl size=9106 sha256=8895a8804dabaf6285e62fdb9a65488ea5d451ce782c87106d7571a6edbb61ff
  Stored in directory: /tmp/pip-ephem-wheel-cache-eld1h2ik/wheels/dd/a3/7d/774508ed192455403be294c8a6c4ad1c83dde8fcdd8903e64a
Successfully built RealESRGAN


# Upscale images

In [2]:
# initialize Real-ERSGAN model
from RealESRGAN import RealESRGAN
from PIL import Image
import numpy as np
import torch

# check c
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model_scale = "2" #@param ["2", "4", "8"] {allow-input: false}

model = RealESRGAN(device, scale=int(model_scale))
model.load_weights(f'weights/RealESRGAN_x{model_scale}.pth')

device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:672: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


RealESRGAN_x2.pth:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Weights downloaded to: weights/RealESRGAN_x2.pth


In [5]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/AdvancedComputerVision/[Augmented]Bike_Helmet_Dataset')
%ls

Mounted at /content/drive
augmented_test/  augmented_train/  augmented_valid/


In [13]:
#@title upscale images or .tar archives
import os
from google.colab import files
from google.colab import output
import shutil
from io import BytesIO
import io
import tarfile
from tqdm import tqdm

IMAGE_FORMATS = ('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')

def image_to_tar_format(img, image_name):
    buff = BytesIO()
    if '.png' in image_name.lower():
        img = img.convert('RGBA')
        img.save(buff, format='PNG')
    else:
        img.save(buff, format='JPEG')
    buff.seek(0)
    fp = io.BufferedReader(buff)
    img_tar_info = tarfile.TarInfo(name=image_name)
    img_tar_info.size = len(buff.getvalue())
    return img_tar_info, fp

def process_tar(path_to_tar, mode):
    # modifying root file name
    if mode == 1:
        temp = 'augmented_train/'
    elif mode == 2:
        temp = 'augmented_valid/'
    elif mode == 3:
        temp = 'augmented_test/'

    os.makedirs(path, exist_ok=True)
    processing_tar = tarfile.open(path_to_tar, mode='r')
    result_tar_path = os.path.join(temp, os.path.basename(path_to_tar))
    save_tar = tarfile.open(result_tar_path, 'w')

    for c, member in enumerate(processing_tar):
        print(f'{c}, processing {member.name}')

        if not member.name.endswith(IMAGE_FORMATS):
            continue

        try:
            img_bytes = BytesIO(processing_tar.extractfile(member.name).read())
            img_lr = Image.open(img_bytes, mode='r').convert('RGB')
        except Exception as err:
            print(f'Unable to open file {member.name}, skipping')
            continue

        img_sr = model.predict(np.array(img_lr))
        # adding to save_tar
        img_tar_info, fp = image_to_tar_format(img_sr, member.name)
        save_tar.addfile(img_tar_info, fp)

    processing_tar.close()
    save_tar.close()
    print(f'Finished! Archive saved to {result_tar_path}')

def process_input(filename, mode):
    # modifying root file name
    if mode == 1:
        temp = 'augmented_train/images'
    elif mode == 2:
        temp = 'augmented_valid/images'
    elif mode == 3:
        temp = 'augmented_test/images'

    result_image_path = os.path.join(temp, os.path.basename(filename))
    image = Image.open(filename).convert('RGB')
    sr_image = model.predict(np.array(image))
    sr_image.save(result_image_path)
    print(f'Finished! Image saved to {result_image_path}')

In [15]:
# specify the folder location
result_folder = 'augmented_train/images'
os.makedirs(result_folder, exist_ok=True)

# locate and augment files
path = '/content/drive/My Drive/AdvancedComputerVision/[Original]Bike_Helmet_Dataset/train/images'
for image_filename in tqdm((os.listdir(path)), desc="Processing:"):
    filename_path = os.path.join(path, image_filename)
    process_input(filename_path, mode=1)
    output.clear()

Processing:: 100%|██████████| 2985/2985 [44:24<00:00,  1.12it/s]


In [16]:
# specify the folder location
result_folder = 'augmented_valid/images'
os.makedirs(result_folder, exist_ok=True)

# locate and augment files
path = '/content/drive/My Drive/AdvancedComputerVision/[Original]Bike_Helmet_Dataset/valid/images'
for image_filename in tqdm((os.listdir(path)), desc="Processing:"):
    filename_path = os.path.join(path, image_filename)
    process_input(filename_path, mode=2)
    output.clear()

Processing:: 100%|██████████| 375/375 [05:32<00:00,  1.13it/s]


In [17]:
# specify the folder location
result_folder = 'augmented_test'
os.makedirs(result_folder, exist_ok=True)

# locate and augment files
path = '/content/drive/My Drive/AdvancedComputerVision/[Original]Bike_Helmet_Dataset/test/images'
for image_filename in tqdm((os.listdir(path)), desc="Processing:"):
    filename_path = os.path.join(path, image_filename)
    process_input(filename_path, mode=3)
    output.clear()

Processing:: 100%|██████████| 375/375 [05:33<00:00,  1.13it/s]
